# Notebook 5: Sensor Fusion Basics for Autonomous Vehicles

**Session 1: AI-based Perception Systems in Autonomous Vehicles**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/milinpatel07/Autonomous-Driving_AI-Safety-and-Security/blob/main/AV_Perception_Safety_Workshop/Session_1_AI_Perception_Systems/notebooks/05_Sensor_Fusion_Basics.ipynb)

**Author:** Milin Patel  
**Duration:** ~15 minutes

---

## 🎯 Learning Objectives

By the end of this notebook, you will:
- ✅ Understand why sensor fusion is critical for AV safety
- ✅ Learn three fusion approaches: early, late, and deep fusion
- ✅ Implement simple late fusion for camera + LiDAR detections
- ✅ Visualize fused detection results
- ✅ Analyze fusion performance vs. single-sensor
- ✅ Understand fusion challenges and failure modes

---

## 📦 Setup and Imports

In [ ]:
import sys

# Check if running on Google Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("🔧 Running on Google Colab - Installing dependencies...\n")
    !pip install -q matplotlib numpy scipy pandas seaborn
    print("✅ Setup complete!\n")
else:
    print("💻 Running locally\n")

print("✅ Environment ready!")

In [ ]:
# Import libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.patches import Rectangle
from scipy.optimize import linear_sum_assignment
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

print("✅ All libraries imported successfully!")
print(f"NumPy version: {np.__version__}")

---

## 1️⃣ Why Sensor Fusion?

**Problem:** No single sensor is perfect!

- **Camera:** Good for classification, bad in rain/night
- **LiDAR:** Accurate 3D, expensive, affected by fog
- **Radar:** All-weather, but low resolution

**Solution:** Combine (fuse) multiple sensors to get the best of all worlds!

### Benefits of Sensor Fusion:
1. **Redundancy:** If one sensor fails, others compensate
2. **Complementary strengths:** Camera sees colors, LiDAR measures depth
3. **Improved accuracy:** Reduces false positives/negatives
4. **All-weather operation:** Some sensor always works
5. **Safety compliance:** Required by ISO 26262 for ASIL-D systems

In [ ]:
# Simulate sensor performance in different conditions
conditions = ['Clear\nDay', 'Rain', 'Fog', 'Night', 'Snow']
camera_acc = [95, 40, 30, 25, 35]
lidar_acc = [95, 75, 50, 95, 60]
radar_acc = [85, 95, 85, 90, 90]
fusion_acc = [98, 90, 75, 92, 80]  # Fusion improves overall

x = np.arange(len(conditions))
width = 0.2

fig, ax = plt.subplots(figsize=(14, 6))
bars1 = ax.bar(x - 1.5*width, camera_acc, width, label='📷 Camera', color='#FF6B6B')
bars2 = ax.bar(x - 0.5*width, lidar_acc, width, label='🌐 LiDAR', color='#4ECDC4')
bars3 = ax.bar(x + 0.5*width, radar_acc, width, label='📡 Radar', color='#45B7D1')
bars4 = ax.bar(x + 1.5*width, fusion_acc, width, label='🔗 Fusion', color='#95E1D3')

ax.set_xlabel('Weather Condition', fontsize=12, fontweight='bold')
ax.set_ylabel('Detection Accuracy (%)', fontsize=12, fontweight='bold')
ax.set_title('Sensor Fusion Improves Robustness Across Conditions', 
             fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(conditions)
ax.legend(fontsize=11, loc='lower right')
ax.set_ylim(0, 100)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for bars in [bars1, bars2, bars3, bars4]:
    for bar in bars:
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height + 1,
               f'{int(height)}', ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

print("\n💡 Key Insight: Fusion never worse than best individual sensor!")
print("   In fact, often BETTER due to complementary information.")

---

## 2️⃣ Three Fusion Approaches

### 1. Early Fusion (Data-Level)
Combine **raw sensor data** before processing.

```
Camera Image + LiDAR Points → Fused Input → Detection Model → Objects
```

**Pros:** Preserves all information  
**Cons:** Complex, computationally expensive, requires precise calibration

---

### 2. Late Fusion (Decision-Level)
Process sensors **independently**, then combine detections.

```
Camera → Detection1 ↘
                      → Fusion → Final Objects
LiDAR → Detection2  ↗
```

**Pros:** Simple, modular, can use existing models  
**Cons:** Information loss, harder to resolve conflicts

**Most common in industry!** (We'll implement this today)

---

### 3. Deep Fusion (Feature-Level)
Fuse **learned features** from neural networks.

```
Camera → CNN Features ↘
                        → Fusion Network → Objects
LiDAR → PointNet Features ↗
```

**Pros:** Learns optimal fusion, state-of-the-art accuracy  
**Cons:** Requires large paired datasets, end-to-end training

**Research frontier!** (Used in modern systems like Tesla FSD, Waymo)

In [ ]:
# Compare fusion approaches
fusion_comparison = pd.DataFrame({
    'Approach': ['Early Fusion', 'Late Fusion', 'Deep Fusion'],
    'Fusion Level': ['Raw Data', 'Detections', 'Features'],
    'Complexity': ['High', 'Low', 'Very High'],
    'Accuracy': ['Good', 'Medium', 'Best'],
    'Computational Cost': ['High', 'Low', 'Very High'],
    'Calibration Sensitivity': ['Very High', 'Medium', 'High'],
    'Modularity': ['Low', 'High', 'Medium'],
    'Industry Use': ['Rare', 'Common', 'Growing']
})

display(fusion_comparison)

print("\n💡 Today we implement Late Fusion (simplest, most practical)")
print("   But deep fusion is the future (BEVFusion, TransFusion, etc.)")

---

## 3️⃣ Implement Simple Late Fusion

**Task:** Fuse camera and LiDAR detections

**Algorithm:**
1. Run camera detector → get bounding boxes
2. Run LiDAR detector → get 3D boxes
3. Project 3D boxes to image plane
4. Match camera and LiDAR detections (IoU-based)
5. Combine matched detections (weighted by confidence)
6. Keep unmatched high-confidence detections

Let's implement step-by-step!

In [ ]:
# Helper functions for late fusion

def compute_iou(box1, box2):
    """
    Compute Intersection over Union (IoU) between two 2D boxes.
    
    Args:
        box1, box2: [x1, y1, x2, y2]
    
    Returns:
        iou: float between 0 and 1
    """
    x1_max = max(box1[0], box2[0])
    y1_max = max(box1[1], box2[1])
    x2_min = min(box1[2], box2[2])
    y2_min = min(box1[3], box2[3])
    
    # Intersection area
    inter_width = max(0, x2_min - x1_max)
    inter_height = max(0, y2_min - y1_max)
    inter_area = inter_width * inter_height
    
    # Union area
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
    union_area = box1_area + box2_area - inter_area
    
    if union_area == 0:
        return 0.0
    
    return inter_area / union_area


def match_detections(camera_dets, lidar_dets, iou_threshold=0.3):
    """
    Match camera and LiDAR detections using Hungarian algorithm.
    
    Args:
        camera_dets: List of dicts with 'bbox', 'confidence', 'class'
        lidar_dets: List of dicts with 'bbox', 'confidence', 'class'
        iou_threshold: Minimum IoU for valid match
    
    Returns:
        matches: List of (camera_idx, lidar_idx) pairs
        unmatched_camera: List of camera indices
        unmatched_lidar: List of lidar indices
    """
    if len(camera_dets) == 0 or len(lidar_dets) == 0:
        return [], list(range(len(camera_dets))), list(range(len(lidar_dets)))
    
    # Compute IoU matrix
    iou_matrix = np.zeros((len(camera_dets), len(lidar_dets)))
    for i, cam_det in enumerate(camera_dets):
        for j, lid_det in enumerate(lidar_dets):
            # Only match same class
            if cam_det['class'] == lid_det['class']:
                iou_matrix[i, j] = compute_iou(cam_det['bbox'], lid_det['bbox'])
    
    # Hungarian algorithm for optimal matching
    row_ind, col_ind = linear_sum_assignment(-iou_matrix)  # Maximize IoU
    
    # Filter by threshold
    matches = []
    matched_cam = set()
    matched_lid = set()
    
    for i, j in zip(row_ind, col_ind):
        if iou_matrix[i, j] >= iou_threshold:
            matches.append((i, j))
            matched_cam.add(i)
            matched_lid.add(j)
    
    unmatched_camera = [i for i in range(len(camera_dets)) if i not in matched_cam]
    unmatched_lidar = [j for j in range(len(lidar_dets)) if j not in matched_lid]
    
    return matches, unmatched_camera, unmatched_lidar


def fuse_detections(camera_dets, lidar_dets, camera_weight=0.5):
    """
    Fuse camera and LiDAR detections using late fusion.
    
    Args:
        camera_dets: List of camera detections
        lidar_dets: List of LiDAR detections
        camera_weight: Weight for camera confidence (0-1)
    
    Returns:
        fused_dets: List of fused detections
    """
    # Match detections
    matches, unmatched_cam, unmatched_lid = match_detections(camera_dets, lidar_dets)
    
    fused = []
    
    # Fuse matched detections (weighted average)
    for cam_idx, lid_idx in matches:
        cam_det = camera_dets[cam_idx]
        lid_det = lidar_dets[lid_idx]
        
        # Average bounding box
        fused_bbox = [
            camera_weight * cam_det['bbox'][i] + (1 - camera_weight) * lid_det['bbox'][i]
            for i in range(4)
        ]
        
        # Combine confidence (average)
        fused_conf = camera_weight * cam_det['confidence'] + (1 - camera_weight) * lid_det['confidence']
        
        fused.append({
            'bbox': fused_bbox,
            'confidence': fused_conf,
            'class': cam_det['class'],
            'source': 'fused'
        })
    
    # Add high-confidence unmatched detections
    for idx in unmatched_cam:
        if camera_dets[idx]['confidence'] > 0.7:  # High confidence threshold
            det = camera_dets[idx].copy()
            det['source'] = 'camera_only'
            fused.append(det)
    
    for idx in unmatched_lid:
        if lidar_dets[idx]['confidence'] > 0.7:
            det = lidar_dets[idx].copy()
            det['source'] = 'lidar_only'
            fused.append(det)
    
    return fused

print("✅ Fusion functions defined!")

---

## 4️⃣ Test Fusion on Simulated Detections

Let's simulate a driving scene with camera and LiDAR detections.

In [ ]:
# Simulate detections from camera and LiDAR
np.random.seed(42)

# Camera detections (good for classification, some false positives)
camera_detections = [
    {'bbox': [100, 200, 250, 350], 'confidence': 0.92, 'class': 'car'},
    {'bbox': [400, 180, 520, 320], 'confidence': 0.88, 'class': 'car'},
    {'bbox': [650, 220, 720, 380], 'confidence': 0.65, 'class': 'pedestrian'},  # Low conf
    {'bbox': [800, 150, 900, 280], 'confidence': 0.45, 'class': 'bicycle'},     # False positive?
]

# LiDAR detections (accurate 3D, may miss small objects)
lidar_detections = [
    {'bbox': [105, 205, 248, 348], 'confidence': 0.95, 'class': 'car'},        # Matches cam[0]
    {'bbox': [395, 185, 525, 325], 'confidence': 0.90, 'class': 'car'},        # Matches cam[1]
    {'bbox': [300, 250, 380, 360], 'confidence': 0.85, 'class': 'truck'},      # Unmatched
    # Note: LiDAR missed the pedestrian (too small/far)
]

print("📷 Camera Detections:")
for i, det in enumerate(camera_detections):
    print(f"   {i+1}. {det['class']} (conf: {det['confidence']:.2f}): {det['bbox']}")

print("\n🌐 LiDAR Detections:")
for i, det in enumerate(lidar_detections):
    print(f"   {i+1}. {det['class']} (conf: {det['confidence']:.2f}): {det['bbox']}")

In [ ]:
# Perform fusion
fused_detections = fuse_detections(camera_detections, lidar_detections, camera_weight=0.5)

print("\n🔗 Fused Detections:")
for i, det in enumerate(fused_detections):
    print(f"   {i+1}. {det['class']} (conf: {det['confidence']:.2f}, source: {det['source']}): {[int(x) for x in det['bbox']]}")

print(f"\n📊 Summary:")
print(f"   Camera: {len(camera_detections)} detections")
print(f"   LiDAR: {len(lidar_detections)} detections")
print(f"   Fused: {len(fused_detections)} detections")
print(f"\n💡 Fusion filtered out low-confidence false positives!")

### Visualize Fusion Results

In [ ]:
# Visualization function
def visualize_detections(camera_dets, lidar_dets, fused_dets, img_size=(1000, 400)):
    """
    Visualize camera, LiDAR, and fused detections side by side.
    """
    fig, axes = plt.subplots(1, 3, figsize=(18, 6))
    
    # Create dummy background
    for ax in axes:
        ax.set_xlim(0, img_size[0])
        ax.set_ylim(img_size[1], 0)  # Flip y-axis for image coordinates
        ax.set_aspect('equal')
        ax.grid(True, alpha=0.3)
    
    # Camera detections
    for det in camera_dets:
        x1, y1, x2, y2 = det['bbox']
        width, height = x2 - x1, y2 - y1
        color = 'green' if det['confidence'] > 0.7 else 'orange' if det['confidence'] > 0.5 else 'red'
        rect = Rectangle((x1, y1), width, height, linewidth=2, 
                        edgecolor=color, facecolor='none')
        axes[0].add_patch(rect)
        axes[0].text(x1, y1-5, f"{det['class']}\n{det['confidence']:.2f}", 
                    fontsize=9, color=color, fontweight='bold')
    axes[0].set_title(f'📷 Camera Detections ({len(camera_dets)})', 
                     fontsize=12, fontweight='bold')
    
    # LiDAR detections
    for det in lidar_dets:
        x1, y1, x2, y2 = det['bbox']
        width, height = x2 - x1, y2 - y1
        color = 'blue'
        rect = Rectangle((x1, y1), width, height, linewidth=2, 
                        edgecolor=color, facecolor='none')
        axes[1].add_patch(rect)
        axes[1].text(x1, y1-5, f"{det['class']}\n{det['confidence']:.2f}", 
                    fontsize=9, color=color, fontweight='bold')
    axes[1].set_title(f'🌐 LiDAR Detections ({len(lidar_dets)})', 
                     fontsize=12, fontweight='bold')
    
    # Fused detections
    for det in fused_dets:
        x1, y1, x2, y2 = det['bbox']
        width, height = x2 - x1, y2 - y1
        if det['source'] == 'fused':
            color = 'purple'
        elif det['source'] == 'camera_only':
            color = 'green'
        else:
            color = 'blue'
        rect = Rectangle((x1, y1), width, height, linewidth=2, 
                        edgecolor=color, facecolor='none', linestyle='--' if 'only' in det['source'] else '-')
        axes[2].add_patch(rect)
        axes[2].text(x1, y1-5, f"{det['class']}\n{det['confidence']:.2f}\n({det['source']})", 
                    fontsize=8, color=color, fontweight='bold')
    axes[2].set_title(f'🔗 Fused Detections ({len(fused_dets)})', 
                     fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

# Visualize
visualize_detections(camera_detections, lidar_detections, fused_detections)

print("\n💡 Notice:")
print("   - Purple boxes: Fused from both sensors (highest confidence)")
print("   - Green dashed: Camera-only (e.g., small objects LiDAR missed)")
print("   - Blue dashed: LiDAR-only (e.g., objects camera misclassified)")
print("   - Low-confidence detections filtered out!")

---

## 5️⃣ Analyze Fusion Performance

Let's compare precision and recall for single-sensor vs. fusion.

In [ ]:
# Simulate ground truth and compute metrics
ground_truth = [
    {'class': 'car', 'bbox': [100, 200, 250, 350]},
    {'class': 'car', 'bbox': [400, 180, 520, 320]},
    {'class': 'truck', 'bbox': [300, 250, 380, 360]},
    {'class': 'pedestrian', 'bbox': [650, 220, 720, 380]},
]

def compute_metrics(detections, ground_truth, iou_threshold=0.5):
    """
    Compute precision and recall.
    """
    true_positives = 0
    
    for det in detections:
        for gt in ground_truth:
            if det['class'] == gt['class']:
                iou = compute_iou(det['bbox'], gt['bbox'])
                if iou >= iou_threshold:
                    true_positives += 1
                    break
    
    precision = true_positives / len(detections) if len(detections) > 0 else 0
    recall = true_positives / len(ground_truth) if len(ground_truth) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return precision, recall, f1

# Compute metrics for each approach
cam_p, cam_r, cam_f1 = compute_metrics(camera_detections, ground_truth)
lid_p, lid_r, lid_f1 = compute_metrics(lidar_detections, ground_truth)
fus_p, fus_r, fus_f1 = compute_metrics(fused_detections, ground_truth)

# Visualize comparison
metrics_df = pd.DataFrame({
    'Method': ['Camera', 'LiDAR', 'Fusion'],
    'Precision': [cam_p, lid_p, fus_p],
    'Recall': [cam_r, lid_r, fus_r],
    'F1-Score': [cam_f1, lid_f1, fus_f1]
})

display(metrics_df)

# Bar chart
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(3)
width = 0.25

ax.bar(x - width, metrics_df['Precision'], width, label='Precision', color='#FF6B6B')
ax.bar(x, metrics_df['Recall'], width, label='Recall', color='#4ECDC4')
ax.bar(x + width, metrics_df['F1-Score'], width, label='F1-Score', color='#95E1D3')

ax.set_ylabel('Score', fontsize=12, fontweight='bold')
ax.set_title('Fusion Improves Detection Performance', fontsize=14, fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(metrics_df['Method'])
ax.legend()
ax.set_ylim(0, 1.0)
ax.grid(True, alpha=0.3, axis='y')

# Add value labels
for i in range(3):
    for j, metric in enumerate(['Precision', 'Recall', 'F1-Score']):
        val = metrics_df.iloc[i][metric]
        ax.text(i + (j-1)*width, val + 0.02, f'{val:.2f}', 
               ha='center', fontsize=9, fontweight='bold')

plt.tight_layout()
plt.show()

print(f"\n📊 Results:")
print(f"   Camera: Precision={cam_p:.2f}, Recall={cam_r:.2f}, F1={cam_f1:.2f}")
print(f"   LiDAR:  Precision={lid_p:.2f}, Recall={lid_r:.2f}, F1={lid_f1:.2f}")
print(f"   Fusion: Precision={fus_p:.2f}, Recall={fus_r:.2f}, F1={fus_f1:.2f}")
print(f"\n✅ Fusion achieves best F1-score by balancing precision and recall!")

---

## 6️⃣ Fusion Challenges and Failure Modes

**Sensor fusion is not a silver bullet!** It has challenges:

### 1. Calibration Errors
- Sensors must be precisely aligned
- Misalignment → wrong associations
- Example: Camera thinks object at X, LiDAR says Y → no match!

### 2. Temporal Synchronization
- Sensors capture at different times
- Fast-moving objects → position mismatch
- Need timestamp alignment (typically 100ms tolerance)

### 3. Conflicting Information
- Camera: "It's a car"
- LiDAR: "It's a truck"
- How to resolve? (Usually: trust higher-confidence sensor)

### 4. Sensor Degradation
- Dirty lens, water droplets, mud
- System must detect degraded sensors
- Fallback to other sensors

### 5. Computational Cost
- Processing multiple sensors is expensive
- Real-time requirement: < 100ms
- Need efficient algorithms + GPUs

In [ ]:
# Simulate calibration error impact
calibration_errors = [0, 10, 20, 30, 40, 50]  # pixels
fusion_accuracy = []

for error in calibration_errors:
    # Shift LiDAR detections to simulate calibration error
    shifted_lidar = []
    for det in lidar_detections:
        shifted = det.copy()
        shifted['bbox'] = [det['bbox'][i] + error for i in range(4)]
        shifted_lidar.append(shifted)
    
    # Fuse with shifted LiDAR
    fused = fuse_detections(camera_detections, shifted_lidar)
    _, _, f1 = compute_metrics(fused, ground_truth)
    fusion_accuracy.append(f1)

# Plot impact
plt.figure(figsize=(10, 6))
plt.plot(calibration_errors, fusion_accuracy, 'o-', linewidth=2, markersize=8, color='#FF6B6B')
plt.xlabel('Calibration Error (pixels)', fontsize=12, fontweight='bold')
plt.ylabel('Fusion F1-Score', fontsize=12, fontweight='bold')
plt.title('Impact of Calibration Error on Fusion Performance', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.axhline(y=cam_f1, color='green', linestyle='--', label='Camera-only baseline')
plt.axhline(y=lid_f1, color='blue', linestyle='--', label='LiDAR-only baseline')
plt.legend(fontsize=11)
plt.tight_layout()
plt.show()

print("\n⚠️ Calibration Error Impact:")
print(f"   0 pixels error: F1 = {fusion_accuracy[0]:.2f}")
print(f"   50 pixels error: F1 = {fusion_accuracy[-1]:.2f}")
print(f"\n💡 With large calibration errors, fusion can be WORSE than single sensor!")
print("   → Regular calibration checks are critical for safety")

---

## ✏️ Exercise: Design a Fusion Strategy

**Scenario:** You're designing sensor fusion for a Level 4 autonomous shuttle.

**Available sensors:**
- 4 cameras (front, back, left, right)
- 1 LiDAR (360°)
- 2 radars (front, back)

**Questions:**
1. Which fusion approach would you use? (early/late/deep)
2. How would you handle sensor failures?
3. What's your strategy for conflicting detections?
4. How would you validate fusion performance?

In [ ]:
# TODO: Fill in your fusion strategy
your_strategy = {
    'fusion_approach': '',  # early/late/deep
    'reasoning': '',
    'failure_handling': '',
    'conflict_resolution': '',
    'validation_plan': ''
}

print("💡 Consider:")
print("   - Computational constraints (real-time)")
print("   - Safety criticality (ASIL-D)")
print("   - Sensor failure modes")
print("   - Environmental conditions")

# Sample answer (uncomment to see)
# sample_answer = {
#     'fusion_approach': 'Late fusion with deep fusion for critical zones',
#     'reasoning': 'Late fusion is proven and modular. Deep fusion for pedestrian-rich areas.',
#     'failure_handling': 'Detect sensor health, degrade gracefully, use redundancy',
#     'conflict_resolution': 'Weighted voting based on sensor confidence + environmental conditions',
#     'validation_plan': 'Test on nuScenes + custom campus data + simulation + field tests'
# }
# print("\nSample Answer:", sample_answer)

---

## 🎯 Key Takeaways

### Why Fusion?
- **No single sensor is perfect** - each has strengths and weaknesses
- **Complementary information** - camera for semantics, LiDAR for 3D, radar for weather
- **Redundancy** - critical for safety (ISO 26262 requirement)

### Three Fusion Approaches
1. **Early fusion:** Fuse raw data (complex, best info preservation)
2. **Late fusion:** Fuse detections (simple, modular, industry standard)
3. **Deep fusion:** Fuse learned features (state-of-the-art, requires large data)

### Fusion Benefits
- ✅ Improved accuracy (higher precision AND recall)
- ✅ Robustness to weather (always have working sensor)
- ✅ Reduced false positives/negatives
- ✅ Safety through redundancy

### Challenges
- ⚠️ Calibration errors degrade fusion
- ⚠️ Temporal synchronization needed
- ⚠️ Computational cost (real-time constraint)
- ⚠️ Conflicting information resolution

### Best Practices
1. **Regular calibration** - check alignment frequently
2. **Timestamp alignment** - synchronize sensor data
3. **Sensor health monitoring** - detect degradation
4. **Graceful degradation** - fallback strategies
5. **Extensive validation** - test all weather/lighting conditions

---

## 🔜 Next: Pedestrian Detection Case Study

Now let's apply what we learned to a safety-critical task: **pedestrian detection**!

**Open Notebook 6:** `06_Pedestrian_Detection_Case_Study.ipynb`

---

*Notebook created by Milin Patel | Hochschule Kempten*  
*Last updated: 2025-01-17*